In [2]:
from models import AlexNet, VGG13

from tensorflow.keras.datasets import cifar10
import tensorflow as tf
import numpy as np

### CIFAR-10

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [11]:
len(x_train), len(x_test)

(50000, 10000)

In [12]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [13]:
vgg13 = VGG13((224,224), 10)

TypeError: 'module' object is not callable